In [2]:
import dataclasses
import functools
import logging
import sys
from collections.abc import Callable

from effectful.handlers.llm import Template
from effectful.handlers.llm.providers import (
    CacheLLMRequestHandler,
    LiteLLMProvider,
    LLMLoggingHandler,
    RetryLLMHandler,
    completion,
    tool_call,
)
from effectful.handlers.llm.synthesis import ProgramSynthesis
from effectful.ops.semantics import fwd, handler
from effectful.ops.syntax import defop

provider = LiteLLMProvider()

## Interface

The `robotl.ops.llm` module provides a simplified LLM interface that uses algebraic effects to provide modularity. The module interface consists of:

- A decorator `template` which creates a prompt template from a callable. We should think of the prompt template as an LLM-implemented function with behavior specified by a template string. When a templated function is called, an LLM is invoked to produce the specified behavior. The `__call__` method of a template is a handleable operation.
- An operation `decode` which parses LLM output. `decode(t: type, c: str)` converts an LLM response `c` to the type `t`. It can be handled to provide decoding logic for particular types.
- Interpretations for LLM providers `OpenAIIntp` and callable decoding `ProgramSynthesisIntp`. These interpretations can be composed to handle a variety of template behaviors.

## Prompt Templates

This template function writes (bad) poetry on a given theme. While difficult to implement in Python, an LLM can provide a reasonable implementation.

In [3]:
@Template.define
def limerick(theme: str) -> str:
    """Write a limerick on the theme of {theme}."""
    raise NotImplementedError

If we call the template with a provider interpretation installed, we get reasonable behavior. The LLM is nondeterministic by default, so calling the template twice with the same arguments gives us different results.

Templates are regular callables, so can be converted to operations with `defop` if we want to override the LLM implementation in some cases.

In [4]:
with handler(provider):
    print(limerick("fish"))
    print("-" * 40)
    print(limerick("fish"))

There once was a fish in a brook,  
Who swam with a curious look.  
With scales shining bright,  
It leapt in delight,  
And was caught on the end of a hook.  
----------------------------------------
In the ocean where fish like to glee,  
Swam a cod with a penchant for tea.  
He brewed by the shore,  
Drinking more and more,  
Till he steeped in a splash of the sea!  


If we want deterministic behavior, we can cache the template call. We can either cache it with the default `@functools.cache` or using `CacheLLMRequestHandler`:

In [5]:
@functools.cache
@Template.define
def haiku(theme: str) -> str:
    """Write a haiku on the theme of {theme}."""
    raise NotImplementedError


@Template.define
def haiku_no_cache(theme: str) -> str:
    """Write a haiku on the theme of {theme}."""
    raise NotImplementedError


print()
with handler(provider):
    print(haiku("fish"))
    print("-" * 40)
    print(haiku("fish"))

print()
cache_handler1 = CacheLLMRequestHandler()
with handler(provider), handler(cache_handler1):
    print(haiku_no_cache("fish2"))
    print("-" * 40)
    print(haiku_no_cache("fish2"))

print()
cache_handler2 = CacheLLMRequestHandler()
with handler(provider), handler(cache_handler2):
    print(haiku_no_cache("fish3"))
    print("-" * 40)
    print(haiku_no_cache("fish3"))


Beneath ripples dance,  
Silent fins weave liquid trails—  
Life in water's grasp.
----------------------------------------
Beneath ripples dance,  
Silent fins weave liquid trails—  
Life in water's grasp.

Fish glide in silence,  
Ripples dance on water's edge,  
Nature's calm embrace.  
----------------------------------------
Fish glide in silence,  
Ripples dance on water's edge,  
Nature's calm embrace.  

Silver scales glisten,  
Underneath the gentle waves,  
Life's dance in the deep.
----------------------------------------
Silver scales glisten,  
Underneath the gentle waves,  
Life's dance in the deep.


## Converting LLM Results to Python Objects

Type conversion is handled by `decode`. By default, primitive types are converted. `DecodeError` is raised if a response cannot be converted.

In [6]:
@Template.define
def primes(first_digit: int) -> int:
    """Give a prime number with {first_digit} as the first digit."""
    raise NotImplementedError


with handler(provider):
    assert type(primes(6)) is int

More complex types can be converted by providing handlers for `decode`. `ProgramSynthesisIntp` provides a `decode` handler that parses Python callables.

In [7]:
@Template.define
def count_char(char: str) -> Callable[[str], int]:
    """Write a function which takes a string and counts the occurrances of '{char}'."""
    raise NotImplementedError


with handler(provider), handler(ProgramSynthesis()):
    count_a = count_char("a")
    assert callable(count_a)
    assert count_a("banana") == 3
    assert count_a("cherry") == 0

## Tool Calling

Passing `Operation`s to `Template.define` makes them available for the LLM to call as tools. The description of these operations is inferred from their type annotations and docstrings.

Tool calls are mediated by a helper operation `tool_call`. Handling this operation allows tool use to be tracked or logged.

In [8]:
@defop
def cities() -> list[str]:
    return ["Chicago", "New York", "Barcelona"]


@defop
def weather(city: str) -> str:
    status = {"Chicago": "cold", "New York": "wet", "Barcelona": "sunny"}
    return status.get(city, "unknown")


@Template.define(tools=[cities, weather])
def vacation() -> str:
    """Use the provided tools to suggest a city that has good weather."""
    raise NotImplementedError


def log_tool_call(_, tool, *args, **kwargs):
    result = fwd()
    print(f"Tool call: {tool}(*{args}, **{kwargs}) -> {result}")
    return result


with handler(provider), handler({tool_call: log_tool_call}):
    print(vacation())

Tool call: cities(*(), **{}) -> ['Chicago', 'New York', 'Barcelona']
Tool call: weather(*(), **{'city': 'Chicago'}) -> cold
Tool call: weather(*(), **{'city': 'New York'}) -> wet
Tool call: weather(*(), **{'city': 'Barcelona'}) -> sunny
Among the cities, Barcelona currently has good weather as it is sunny.


## Structured Output Generation

Constrained generation is used for any type that is convertible to a Pydantic model.

In [9]:
@dataclasses.dataclass
class KnockKnockJoke:
    whos_there: str
    punchline: str


@Template.define
def write_joke(theme: str) -> KnockKnockJoke:
    """Write a knock-knock joke on the theme of {theme}."""
    raise NotImplementedError


@Template.define
def rate_joke(joke: KnockKnockJoke) -> bool:
    """Decide if {joke} is funny or not"""
    raise NotImplementedError


def do_comedy():
    joke = write_joke("lizards")
    print("> You are onstage at a comedy club. You tell the following joke:")
    print(
        f"Knock knock.\nWho's there?\n{joke.whos_there}.\n{joke.whos_there} who?\n{joke.punchline}"
    )
    if rate_joke(joke):
        print("> The crowd laughs politely.")
    else:
        print("> The crowd stares in stony silence.")


with handler(provider):
    do_comedy()

> You are onstage at a comedy club. You tell the following joke:
Knock knock.
Who's there?
Iguana.
Iguana who?
Iguana who? Iguana come in and hang out on your wall, like a real lizard!
> The crowd laughs politely.


### Logging LLM requests
To intercept messages being called on the lower-level, we can write a handler for `completion`:

In [10]:
def log_llm(*args, **kwargs):
    result = fwd()
    print("Request fired: ", args, kwargs, result)
    return result


# Avoid cache
try:
    haiku.cache_clear()
except Exception:
    pass

# Put completion handler innermost so it has highest precedence during the call
with handler(provider), handler({completion: log_llm}):
    _ = haiku("fish2")
    _ = limerick("fish")  # or use haiku("fish-2") to avoid cache

Request fired:  () {'messages': [{'type': 'message', 'content': [{'type': 'text', 'text': 'Write a haiku on the theme of fish2.'}], 'role': 'user'}], 'response_format': None, 'tools': []} ModelResponse(id='chatcmpl-Cjnw6XOyfwR7aFre43pXB7EvzDC14', created=1765032750, model='gpt-4o-2024-08-06', object='chat.completion', system_fingerprint='fp_83554c687e', choices=[Choices(finish_reason='stop', index=0, message=Message(content='In clear streams they glide,  \nSilvery scales in sunlight,  \nSilent depths they hide.  ', role='assistant', tool_calls=None, function_call=None, provider_specific_fields={'refusal': None}, annotations=[]), provider_specific_fields={})], usage=Usage(completion_tokens=21, prompt_tokens=34, total_tokens=55, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0, text_tokens=None, image_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_

### Python logging for LLM requests and tool calls
We can also uses Python logger through `LLMLoggingHandler` to log both low-level LLM requests (`completion`) and model-initiated tool use (`tool_call`):


In [11]:
# 1. Create a logger
logger = logging.getLogger("effectful.llm")
logger.setLevel(logging.INFO)
log_handler = logging.StreamHandler(sys.stdout)
log_handler.setFormatter(logging.Formatter("%(levelname)s %(payload)s"))
logger.addHandler(log_handler)
# 2. Pass it to the handler
llm_logger = LLMLoggingHandler(logger=logger)  # can also be LLMLoggingHandler()

# Avoid cache for demonstration
try:
    haiku.cache_clear()
    limerick.cache_clear()
except Exception:
    pass

with handler(provider), handler(llm_logger):
    _ = haiku("fish3")
    _ = limerick("fish4")

INFO {'args': (), 'kwargs': {'messages': [{'type': 'message', 'content': [{'type': 'text', 'text': 'Write a haiku on the theme of fish3.'}], 'role': 'user'}], 'response_format': None, 'tools': []}, 'response': ModelResponse(id='chatcmpl-Cjnw9IbWFpdT1zuT7GOkkf31e3ClW', created=1765032753, model='gpt-4o-2024-08-06', object='chat.completion', system_fingerprint='fp_83554c687e', choices=[Choices(finish_reason='stop', index=0, message=Message(content='Gentle ripples dance,  \nBeneath the tranquil surface,  \nFish in silent glide.', role='assistant', tool_calls=None, function_call=None, provider_specific_fields={'refusal': None}, annotations=[]), provider_specific_fields={})], usage=Usage(completion_tokens=21, prompt_tokens=34, total_tokens=55, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0, text_tokens=None, image_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_token

### Retrying LLM Requests
LLM calls can sometimes fail due to transient errors or produce invalid outputs. The `RetryLLMHandler` automatically retries failed template calls:

- `max_retries`: Maximum number of retry attempts (default: 3)
- `add_error_feedback`: When `True`, appends the error message to the prompt on retry, helping the LLM correct its output


Example usage: having an unstable service that seldomly fail.

In [ ]:
call_count = 0
REQUIRED_RETRIES = 3


def unstable_service() -> str:
    """Fetch data from an unstable external service. May require retries."""
    global call_count
    call_count += 1
    if call_count < REQUIRED_RETRIES:
        raise ConnectionError(
            f"Service unavailable! Attempt {call_count}/{REQUIRED_RETRIES}. Please retry."
        )
    return "{ 'status': 'ok', 'data': [1, 2, 3] }"


@Template.define(tools=[unstable_service])
def fetch_data() -> str:
    """Use the unstable_service tool to fetch data."""
    raise NotImplementedError


retry_handler = RetryLLMHandler(max_retries=5, add_error_feedback=True)

with handler(provider), handler(retry_handler), handler(llm_logger):
    result = fetch_data()
    print(f"Result: {result}", "Retries:", call_count)

INFO {'args': (), 'kwargs': {'messages': [{'type': 'message', 'content': [{'type': 'text', 'text': 'Use the unstable_service tool to fetch data.'}], 'role': 'user'}], 'response_format': None, 'tools': [{'type': 'function', 'function': {'name': 'unstable_service', 'description': 'Fetch data from an unstable external service. May require retries.', 'parameters': {'additionalProperties': False, 'properties': {}, 'title': 'Params', 'type': 'object', 'required': []}, 'strict': True}}]}, 'response': ModelResponse(id='chatcmpl-CjnzED6zrWNc9JkHY4XRKQKOAkV1T', created=1765032944, model='gpt-4o-2024-08-06', object='chat.completion', system_fingerprint='fp_83554c687e', choices=[Choices(finish_reason='tool_calls', index=0, message=Message(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(function=Function(arguments='{}', name='unstable_service'), id='call_qnx354HC40IvPkMJYxhCAwCK', type='function')], function_call=None, provider_specific_fields={'refusal': None}, annotation